In [4]:
import cv2
import glob



In [21]:
img_array = []
for filename in glob.glob('train/train_sharp_bicubic/X4/000/*.png'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width, height)
    img_array.append(img)

out = cv2.VideoWriter('video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 5, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()


In [ ]:
import cv2

cam = cv2.VideoCapture("video.mp4")

currentframe = 0

while (True):
    # reading from frame
    ret, frame = cam.read()

    if ret:
        name = './data/frame' + str(currentframe) + '.jpg'
        print('Creating...' + name)

        cv2.imwrite(name, frame)

        currentframe += 1
    else:
        break

cam.release()
cv2.destroyAllWindows()

In [25]:
import torch
import torch_enhance.models.srresnet as srresnet

model = srresnet.SRResNet(scale_factor=2, channels=3)
model.eval()
video_path = 'video.mp4'
cap = cv2.VideoCapture(video_path)
out = cv2.VideoWriter('video2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 15, size)

frame_tensor = torch.from_numpy(img_array[0]).permute(2, 0, 1)
sr_img = model(frame_tensor)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
# 
#     frame_tensor = torch.from_numpy(frame).permute(2, 0, 1)
# 
#     sr_frame = model(frame_tensor)
# 
#     sr_frame = sr_frame.permute(1, 2, 0).numpy()
# 
#     out.write(sr_frame)

cap.release()
out.release()
cv2.destroyAllWindows()

RuntimeError: Input type (unsigned char) and bias type (float) should be the same

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch_enhance.models.srresnet import SRResNet
from your_dataset_module import YourDataset 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 10
batch_size = 16
learning_rate = 0.001

sr_resnet_model = SRResNet(scale_factor=2, channels=3).to(device)

criterion = nn.MSELoss()

optimizer = optim.Adam(sr_resnet_model.parameters(), lr=learning_rate)

train_dataset = YourDataset(root_dir='train/train_blur_bicubic/X4/000', transform=ToTensor())

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
for epoch in range(num_epochs):
    for batch in train_loader:
        low_res_images, high_res_images = batch[0].to(device), batch[1].to(device)

        optimizer.zero_grad()

        outputs = sr_resnet_model(low_res_images)

        loss = criterion(outputs, high_res_images)

        loss.backward()

        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')